In [46]:
#получение обучающей, тестовой и тестовой выборки
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.models import load_model
from keras import backend as K
import joblib

In [291]:
loaded_model_away_poly = joblib.load('model_poly_away_1.joblib')
loaded_model_home_poly = joblib.load('model_poly_home_1.joblib')


In [292]:
scaler_filename_home = "scaler_home.save"
scaler_filename_away = "scaler_away.save"
scaler_home = joblib.load(scaler_filename_home)
scaler_away = joblib.load(scaler_filename_away)

In [293]:
loaded_model_home_neuron = load_model('home_model_1.h5')
loaded_model_away_neuron = load_model('away_model_1.h5')

In [294]:
match_data = pd.read_csv('match_data.csv',index_col='id')

In [295]:
match_data['points'] = match_data.apply(lambda row: 3 if row['goals'] > match_data[(match_data['fixtureID'] == row['fixtureID']) & (match_data['team_id'] != row['team_id'])]['goals'].max() else (0 if row['goals'] < match_data[(match_data['fixtureID'] == row['fixtureID']) & (match_data['team_id'] != row['team_id'])]['goals'].max() else 1), axis=1)

In [296]:
def create_samples(data):
    x = []
    for _, row in data.iterrows():
        new_line = []
        team_data = match_data[(match_data['team_id'] == row['team_id']) & (match_data['time'] < row['time'])][-10::]
        home_data = team_data[team_data['position'] == 'home']
        away_data = team_data[team_data['position'] == 'away']
        points_last = match_data[(match_data['team_id'] == row['team_id']) & (match_data['time'] < row['time'])][-5::]['points'].sum()
        print(match_data[(match_data['team_id'] == row['team_id']) & (match_data['time'] < row['time'])][-5::])
        new_line.extend([row['team_id'], row['name'] ,row['time'], home_data['goals'].mean(), away_data['goals'].mean(), home_data['shots'].mean(), away_data['shots'].mean(), home_data['xg'].mean(), away_data['xg'].mean(), points_last])

        x.extend([new_line]) 
        #y.extend([[1, row['home_goals']], [2, row['away_goals']]])
    return x

In [235]:
import http.client
import json

conn = http.client.HTTPSConnection("v3.football.api-sports.io")

headers = {
    'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': "55d8d0822bdb3026f52b479e85e3a447"
    }

conn.request("GET", f"/fixtures?next=8&league=235", headers=headers)
res = conn.getresponse()
data = res.read()
json_data = json.loads(data)
conn.request("GET", f"/fixtures?next=10&league=39", headers=headers)
res = conn.getresponse()
data = res.read()
json_data_2 = json.loads(data)
if 'response' in json_data and json_data['response']:
    json_data = json_data['response']

if 'response' in json_data_2 and json_data_2['response']:
    json_data_2 = json_data_2['response']

json_data = json_data + json_data_2


In [237]:
home_teams = []
away_teams = []
for fixture in json_data:
    home_teams.append([fixture['teams']['home']['id'], fixture['fixture']['timestamp'], fixture['teams']['home']['name']])
    away_teams.append([fixture['teams']['away']['id'], fixture['fixture']['timestamp'], fixture['teams']['away']['name']])
home_teams = pd.DataFrame(home_teams, columns=['team_id', 'time', 'name'])
away_teams = pd.DataFrame(away_teams, columns=['team_id', 'time', 'name'])
home_teams

,team_id,time,name
0,1079,1700906400,Krylya Sovetov
1,596,1700914500,Zenit Saint Petersburg
2,555,1700922600,CSKA Moscow
3,2006,1700930700,Baltika
4,1080,1700989200,Orenburg
5,1084,1700996400,Ural
6,1083,1701005400,Rubin
7,779,1701014400,FC Rostov
8,50,1700915400,Manchester City
9,44,1700924400,Burnley


In [238]:
away_teams

,team_id,time,name
0,597,1700906400,Lokomotiv Moscow
1,2012,1700914500,PFC Sochi
2,1088,1700922600,Dinamo Moscow
3,558,1700930700,Spartak Moscow
4,1085,1700989200,Akhmat Grozny
5,621,1700996400,Krasnodar
6,1993,1701005400,Fakel Voronezh
7,2011,1701014400,Nizhny Novgorod
8,40,1700915400,Liverpool
9,48,1700924400,West Ham


In [297]:
#формирование x
x_home = create_samples(home_teams)
x_home = pd.DataFrame(x_home, columns=['team_id','name', 'time', 'mean_goals_home', 'mean_goals_away', 'shots_home', 'shots_away', 'xg_home', 'xg_away', 'points'])
x_away = create_samples(away_teams)
x_away = pd.DataFrame(x_away, columns=['team_id','name','time', 'mean_goals_home', 'mean_goals_away', 'shots_home', 'shots_away', 'xg_home', 'xg_away', 'points'])
x_home

      fixtureID  team_id position  goals  shots    xg        time  points
id                                                                       
2169    1042102     1079     away    1.0    1.0  1.00  1697981400       0
2184    1042112     1079     home    1.0    5.0  1.00  1698499800       1
2213    1060685     1079     away    0.0    0.0  0.00  1698769800       0
2255    1042118     1079     away    1.0    2.0  0.87  1699182000       0
2272    1042128     1079     home    3.0    9.0  3.00  1699700400       3
      fixtureID  team_id position  goals  shots   xg        time  points
id                                                                      
2168    1042102      596     home    3.0    6.0  3.0  1697981400       3
2192    1042110      596     home    2.0    3.0  2.0  1698508800       3
2212    1060685      596     home    1.0    1.0  1.0  1698769800       3
2251    1042123      596     away    2.0    4.0  2.0  1699113600       3
2286    1042126      596     home    1.0    

,team_id,name,time,mean_goals_home,mean_goals_away,shots_home,shots_away,xg_home,xg_away,points
0,1079,Krylya Sovetov,1700906400,2.500000,1.000000,6.250000,2.166667,2.500000,0.996667,4
1,596,Zenit Saint Petersburg,1700914500,1.666667,2.000000,3.833333,6.500000,1.666667,2.000000,13
2,555,CSKA Moscow,1700922600,1.833333,1.500000,4.500000,3.250000,1.896667,1.500000,11
3,2006,Baltika,1700930700,1.500000,0.250000,3.000000,1.750000,1.500000,0.250000,5
4,1080,Orenburg,1700989200,1.250000,0.833333,5.250000,4.000000,1.250000,0.833333,2
5,1084,Ural,1700996400,0.600000,1.000000,4.800000,3.000000,0.578000,1.000000,0
6,1083,Rubin,1701005400,0.400000,1.000000,2.600000,3.400000,0.400000,1.000000,10
7,779,FC Rostov,1701014400,2.200000,1.200000,5.200000,3.600000,2.200000,1.200000,10
8,50,Manchester City,1700915400,3.750000,1.833333,5.750000,7.666667,1.572500,1.976667,10
9,44,Burnley,1700924400,0.333333,0.714286,4.000000,2.714286,0.990000,0.657143,0


In [298]:
x_away

,team_id,name,time,mean_goals_home,mean_goals_away,shots_home,shots_away,xg_home,xg_away,points
0,597,Lokomotiv Moscow,1700906400,1.666667,1.000000,4.166667,5.250000,1.811667,1.080000,8
1,2012,PFC Sochi,1700914500,1.250000,1.000000,6.000000,3.166667,1.250000,1.428333,7
2,1088,Dinamo Moscow,1700922600,1.800000,1.000000,6.400000,6.800000,1.800000,1.000000,11
3,558,Spartak Moscow,1700930700,1.800000,0.800000,6.600000,2.400000,1.800000,0.738000,7
4,1085,Akhmat Grozny,1700989200,1.200000,0.600000,5.000000,3.000000,1.200000,0.600000,0
5,621,Krasnodar,1700996400,2.200000,1.600000,5.800000,5.800000,2.252000,1.600000,8
6,1993,Fakel Voronezh,1701005400,1.250000,0.666667,4.000000,3.666667,1.250000,0.666667,10
7,2011,Nizhny Novgorod,1701014400,1.200000,0.600000,3.800000,1.400000,1.046000,0.600000,4
8,40,Liverpool,1700915400,2.800000,1.800000,7.000000,5.400000,2.526000,2.206000,13
9,48,West Ham,1700924400,1.833333,1.500000,4.333333,3.250000,1.616667,1.277500,6


In [299]:
elo_england_22 = pd.read_csv('./elo/csv/England/2022/ratingElo2022_2023PL.csv')
elo_england_23 = pd.read_csv('./elo/csv/England/2023/ratingElo2023_2024PL.csv')

elo_england_22.set_index('id', inplace=True)
elo_england_23.set_index('id', inplace=True)

elo_england_23['prev_elos'] = elo_england_23.apply(lambda row: elo_england_22.loc[row.name]['prev_elos'] + row['prev_elos'] if row.name in elo_england_22.index else row['prev_elos'], axis=1)
elo_england_23.reset_index(inplace=True)

elo_russian_22 = pd.read_csv('./elo/csv/Russia/2022/ratingElo2022_2023RFPL.csv')
elo_russian_23 = pd.read_csv('./elo/csv/Russia/2023/ratingElo2023_2024RFPL.csv')

elo_russian_22.set_index('id', inplace=True)
elo_russian_23.set_index('id', inplace=True)

elo_russian_23['prev_elos'] = elo_russian_23.apply(lambda row: elo_russian_22.loc[row.name]['prev_elos'] + row['prev_elos'] if row.name in elo_russian_22.index else row['prev_elos'], axis=1)
elo_russian_23.reset_index(inplace=True)
all_elo = pd.concat([elo_england_23, elo_russian_23])
all_elo.reset_index(inplace=True)

In [242]:
import re, json
def process_row(row):
    return int(all_elo[all_elo['id'] == row['team_id']]['elo_rating'])
def add_column_elo(frame):
    frame['elo'] = frame.apply(process_row, axis=1)

In [300]:
samples = [x_home, x_away]
for item in samples:
    add_column_elo(item)

C:\Users\arbol\AppData\Local\Temp\ipykernel_18136\1540769638.py:3: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  return int(all_elo[all_elo['id'] == row['team_id']]['elo_rating'])
C:\Users\arbol\AppData\Local\Temp\ipykernel_18136\1540769638.py:3: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  return int(all_elo[all_elo['id'] == row['team_id']]['elo_rating'])


In [301]:
x_home['diff_elo'] = x_home['elo'] - x_away['elo']
x_away['diff_elo'] = x_away['elo'] - x_home['elo']

In [302]:
x_home

,team_id,name,time,mean_goals_home,mean_goals_away,shots_home,shots_away,xg_home,xg_away,points,elo,diff_elo
0,1079,Krylya Sovetov,1700906400,2.500000,1.000000,6.250000,2.166667,2.500000,0.996667,4,1584,-128
1,596,Zenit Saint Petersburg,1700914500,1.666667,2.000000,3.833333,6.500000,1.666667,2.000000,13,1826,334
2,555,CSKA Moscow,1700922600,1.833333,1.500000,4.500000,3.250000,1.896667,1.500000,11,1746,59
3,2006,Baltika,1700930700,1.500000,0.250000,3.000000,1.750000,1.500000,0.250000,5,1568,-79
4,1080,Orenburg,1700989200,1.250000,0.833333,5.250000,4.000000,1.250000,0.833333,2,1568,34
5,1084,Ural,1700996400,0.600000,1.000000,4.800000,3.000000,0.578000,1.000000,0,1498,-294
6,1083,Rubin,1701005400,0.400000,1.000000,2.600000,3.400000,0.400000,1.000000,10,1606,14
7,779,FC Rostov,1701014400,2.200000,1.200000,5.200000,3.600000,2.200000,1.200000,10,1608,84
8,50,Manchester City,1700915400,3.750000,1.833333,5.750000,7.666667,1.572500,1.976667,10,2162,89
9,44,Burnley,1700924400,0.333333,0.714286,4.000000,2.714286,0.990000,0.657143,0,1700,-172


In [303]:
x_home_predict = x_home.drop(['time', 'elo', 'name'], axis=1)
x_away_predict = x_away.drop(['time', 'elo', 'name'], axis=1)

In [304]:
x_home_predict = scaler_home.transform(x_home_predict)
x_away_predict = scaler_away.transform(x_away_predict)

In [305]:
poly_home_value = loaded_model_home_poly.predict(x_home_predict)
poly_away_value = loaded_model_away_poly.predict(x_away_predict)

In [309]:
print("Полином:")
for index, row in x_home.iterrows():
    print(f"{row['name']} - {x_away.iloc[index]['name']}: {round(poly_home_value[index], 3)}:{round(poly_away_value[index], 3)}")

Полином:
Krylya Sovetov - Lokomotiv Moscow: 1.011:1.416
Zenit Saint Petersburg - PFC Sochi: 1.564:0.809
CSKA Moscow - Dinamo Moscow: 1.285:1.113
Baltika - Spartak Moscow: 1.081:1.408
Orenburg - Akhmat Grozny: 1.028:1.007
Ural - Krasnodar: 0.771:1.715
Rubin - Fakel Voronezh: 0.854:1.063
FC Rostov - Nizhny Novgorod: 1.321:0.985
Manchester City - Liverpool: 1.434:1.445
Burnley - West Ham: 0.96:1.507
Luton - Crystal Palace: 0.885:1.358
Newcastle - Chelsea: 1.651:1.552
Nottingham Forest - Brighton: 1.01:1.423
Sheffield Utd - Bournemouth: 1.136:1.247
Brentford - Arsenal: 1.19:1.435
Tottenham - Aston Villa: 1.17:1.451
Everton - Manchester United: 1.125:1.329
Fulham - Wolves: 1.211:1.182


In [307]:
nueron_home_value = loaded_model_home_neuron.predict(x_home_predict)
neuron_away_value = loaded_model_away_neuron.predict(x_away_predict)

1/1 [==============================] - 0s 143ms/step


In [308]:
print("Нейронная сеть:")
for index, row in x_home.iterrows():
    home_goals = round(float(nueron_home_value[index][0]),3)
    away_goals = round(float(neuron_away_value[index][0]), 3)
    print(f"{row['name']} - {x_away.iloc[index]['name']}: {home_goals}:{away_goals}")

Нейронная сеть:
Krylya Sovetov - Lokomotiv Moscow: 1.261:1.405
Zenit Saint Petersburg - PFC Sochi: 2.138:1.049
CSKA Moscow - Dinamo Moscow: 1.541:1.311
Baltika - Spartak Moscow: 1.023:1.389
Orenburg - Akhmat Grozny: 1.55:1.235
Ural - Krasnodar: 1.079:1.622
Rubin - Fakel Voronezh: 1.348:1.138
FC Rostov - Nizhny Novgorod: 1.592:1.086
Manchester City - Liverpool: 2.039:1.503
Burnley - West Ham: 1.162:1.472
Luton - Crystal Palace: 1.147:1.368
Newcastle - Chelsea: 1.938:1.397
Nottingham Forest - Brighton: 1.245:1.468
Sheffield Utd - Bournemouth: 1.283:1.324
Brentford - Arsenal: 1.424:1.505
Tottenham - Aston Villa: 1.678:1.463
Everton - Manchester United: 1.38:1.365
Fulham - Wolves: 1.514:1.297


In [271]:
loaded_model_away_neuron